# "Beer Recipes" parte #1 

## Obteniendo estimados de locación y variabilidad

### Objetivo

- Utilizar estimados de locación y variabilidad para describir las columnas numéricas de un dataset.

Este es un conjunto de datos de 75,000 cervezas caseras con más de 176 estilos diferentes enviadas por usuarios extraídas de Brewer's Friend. Los registros de cerveza son informados por el usuario y se clasifican según uno de los 176 estilos diferentes. Sería interesante ver si los datos proporcionados son suficientes para definir cada clase o si existen patrones por descubrir. En el futuro, podría ser posible revisar y extraer información más detallada para cada receta, como la levadura y los lúpulos específicos utilizados.

Link del dataset: 
https://www.kaggle.com/jtrofe/beer-recipes

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
df = pd.read_csv('../Beer Recipe/recipeData.csv', index_col=0)
pd.options.display.max_columns = 0
df.head()

,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
BeerID,,,,,,,,,,,,,,,,,,,,,,
1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0


### A continuación tenemos lo que significa cada variable

##### BeerID
ID de registro

##### Name
Nombre

##### URL
Ubicación de la página web de recetas en https://www.brewersfriend.com

##### Style
Tipo de infusión

##### StyleID
ID numérico para el tipo de preparación

##### Size(L)
Cantidad elaborada para la receta indicada

##### OG
Gravedad específica del mosto antes de la fermentación.

##### FG
Gravedad específica del mosto después de la fermentación.

##### ABV
Alcohol por volumen

##### IBU
Unidades amargas internacionales

##### Color
Método de referencia estándar - claro a oscuro ej. 40 = negro

##### BoilSize
Líquido al comienzo del hervor

##### BoilTime
Tiempo que se hierve el mosto

##### BoilGravity
Gravedad específica del mosto antes del hervor

##### Efficiency
Eficiencia de extracción de puré de cerveza: extracción de azúcares del grano durante el puré

##### MashThickness
Cantidad de agua por libra de grano

##### SugarScale
Escala para determinar la concentración de sólidos disueltos en el mosto

##### BrewMethod
Varias técnicas para elaborar cerveza.

##### PitchRate
Levadura añadida al fermentador por unidad de gravedad - M células / ml / deg P

##### PrimaryTemp
Temperatura en la etapa de fermentación

##### PrimingMethod
N

##### PrimingAmount
Cantidad de azúcar de imprimación utilizada

##### UserId

# Limpieza de Datos

### A continuación debemos revisar los datos nulos del dataset para determinar que haremos con ellos.

In [4]:
df.isna().sum()  # Checamos los datos Nulos

Name                 1
URL                  0
Style              596
StyleID              0
Size(L)              0
OG                   0
FG                   0
ABV                  0
IBU                  0
Color                0
BoilSize             0
BoilTime             0
BoilGravity       2990
Efficiency           0
MashThickness    29864
SugarScale           0
BrewMethod           0
PitchRate        39252
PrimaryTemp      22662
PrimingMethod    67095
PrimingAmount    69087
UserId           50490
dtype: int64

### Primero eliminaremos la columna UserId debido a que contiene muchos datos nulos y no son tan relevantes

In [8]:
df_beer = df.drop(columns=['UserId']) #Eliminamos la columna UserId
df_beer.head()

,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount
BeerID,,,,,,,,,,,,,,,,,,,,,
1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz
2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN
3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN
4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN
5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l


In [10]:
df_beer['BoilGravity']

BeerID
1        1.038
2        1.070
3          NaN
4          NaN
5        1.050
         ...  
73857    1.060
73858    1.034
73859    1.041
73860    1.043
73861    1.056
Name: BoilGravity, Length: 73861, dtype: float64

In [12]:
df_beer['BoilGravity'].mean()

1.35395474876889

In [13]:
promedio1 = 1.35

In [22]:
df_beer['BoilGravity'] = df_beer['BoilGravity'].fillna(promedio1)
df_beer

,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount
BeerID,,,,,,,,,,,,,,,,,,,,,
1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz
2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN
3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.350,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN
4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.350,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN
5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73857,Amber Alfie 2,/homebrew/recipe/view/609673/amber-alfie-2,British Strong Ale,36,21.00,1.052,1.012,5.22,39.73,6.96,18.00,60,1.060,63.0,3.0,Specific Gravity,All Grain,1.25,20.00,NaN,NaN
73858,Rye IPA,/homebrew/recipe/view/610955/rye-ipa,Specialty IPA: Rye IPA,151,10.00,1.059,1.010,6.37,59.21,7.90,17.00,60,1.034,60.0,NaN,Specific Gravity,BIAB,0.50,22.00,NaN,NaN
73859,SK-K�lsch,/homebrew/recipe/view/586891/,NaN,111,16.00,1.046,1.008,5.01,25.42,3.63,18.00,90,1.041,65.0,NaN,Specific Gravity,BIAB,0.35,17.00,sucrose,140 g


Por ultimo eliminaremos las columnas MashThickness, PitchRate, PrimaryTemp, PrimingMethod, PrimingAmount

In [25]:
beer = df_beer.drop(columns=['MashThickness', 'PitchRate', 'PrimaryTemp', 'PrimingMethod', 'PrimingAmount'])
beer.head()

,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,SugarScale,BrewMethod
BeerID,,,,,,,,,,,,,,,,
1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,Specific Gravity,All Grain
2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,Specific Gravity,All Grain
3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.350,70.0,Specific Gravity,extract
4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.350,70.0,Specific Gravity,All Grain
5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,Specific Gravity,All Grain


In [26]:
beer.isna().sum()  # Checamos los datos Nulos

Name             1
URL              0
Style          596
StyleID          0
Size(L)          0
OG               0
FG               0
ABV              0
IBU              0
Color            0
BoilSize         0
BoilTime         0
BoilGravity      0
Efficiency       0
SugarScale       0
BrewMethod       0
dtype: int64

In [27]:
beer_nona = beer.dropna()  #Reemplazaremos los datos nulos con un 0
beer_nona

,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,SugarScale,BrewMethod
BeerID,,,,,,,,,,,,,,,,
1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,Specific Gravity,All Grain
2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,Specific Gravity,All Grain
3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.350,70.0,Specific Gravity,extract
4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.350,70.0,Specific Gravity,All Grain
5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,Specific Gravity,All Grain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73856,Unicorn Pee,/homebrew/recipe/view/607368/unicorn-pee,American IPA,7,22.71,1.065,1.014,6.71,49.35,4.73,15.14,30,1.098,55.0,Specific Gravity,Partial Mash
73857,Amber Alfie 2,/homebrew/recipe/view/609673/amber-alfie-2,British Strong Ale,36,21.00,1.052,1.012,5.22,39.73,6.96,18.00,60,1.060,63.0,Specific Gravity,All Grain
73858,Rye IPA,/homebrew/recipe/view/610955/rye-ipa,Specialty IPA: Rye IPA,151,10.00,1.059,1.010,6.37,59.21,7.90,17.00,60,1.034,60.0,Specific Gravity,BIAB


In [28]:
beer_nona.isna().sum()

Name           0
URL            0
Style          0
StyleID        0
Size(L)        0
OG             0
FG             0
ABV            0
IBU            0
Color          0
BoilSize       0
BoilTime       0
BoilGravity    0
Efficiency     0
SugarScale     0
BrewMethod     0
dtype: int64

In [29]:
beer_nona.to_csv('../Beer Recipe/recipeData_test.csv')  # Creamos nuestro nuevo dataframe con el cual vamos a trabajar

# Estimados de Locación

In [30]:
beers = pd.read_csv('../Beer Recipe/recipeData_test.csv', index_col=0)
beers.head()

,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,SugarScale,BrewMethod
BeerID,,,,,,,,,,,,,,,,
1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,Specific Gravity,All Grain
2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,Specific Gravity,All Grain
3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.350,70.0,Specific Gravity,extract
4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.350,70.0,Specific Gravity,All Grain
5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,Specific Gravity,All Grain


## Media o Promedio

Obtendremos el promedio de la variable abv que es el nivel de alcohol siendo 0 nada de alcohol y 1 alcohol puro

In [31]:
beers['ABV'].mean()

6.13851987333479

La variable ibu representa lo amarga que es la cerveza, obtendremos su promedio también

In [32]:
beers['IBU'].mean()

44.381082113998694

## Mediana

In [33]:
beers['ABV'].median()

5.79

In [34]:
beers['IBU'].median()

35.87

## Media Truncada

In [35]:
mt = stats.trim_mean(beers['IBU'], 0.1)
mt

39.43101071452945

In [36]:
p = abs(beers['IBU'].mean() - mt)

In [37]:
m = abs(beers['IBU'].median() - mt)

In [38]:
print("Distancia promedio: " + str(p))
print("Distancia mediana: "   + str(m))

if (p > m):
    print("La mediana es más cercana")
else:
    print("El promedio es más cercano")

Distancia promedio: 4.950071399469245
Distancia mediana: 3.561010714529452
La mediana es más cercana


## Desviación Estandar

In [39]:
beers['IBU'].std()

41.60689314292866

## Rango

In [40]:
beers['IBU'].max() - beers['IBU'].min()

3409.3

## Percentiles

In [41]:
print(f'Valor mínimo: {beers["IBU"].min()}')
print(f'Percentil 0: {beers["IBU"].quantile(0)}')
print(f'Percentil 10: {beers["IBU"].quantile(0.1)}')
print(f'Percentil 25: {beers["IBU"].quantile(0.25)}')
print(f'Percentil 50: {beers["IBU"].median()}')
print(f'Mediana: {beers["IBU"].quantile(0.5)}')
print(f'Percentil 75: {beers["IBU"].quantile(0.75)}')
print(f'Percentil 90: {beers["IBU"].quantile(0.9)}')
print(f'Percentil 100: {beers["IBU"].quantile(1)}')
print(f'Valor máximo: {beers["IBU"].max()}')

Valor mínimo: 0.0
Percentil 0: 0.0
Percentil 10: 13.73
Percentil 25: 23.49
Percentil 50: 35.87
Mediana: 35.87
Percentil 75: 56.49
Percentil 90: 82.74699999999997
Percentil 100: 3409.3
Valor máximo: 3409.3


## Rango Intercuantilico

In [42]:
beers["IBU"].quantile(0.75) - beers["IBU"].quantile(0.25)

33.0

In [43]:
beers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73264 entries, 1 to 73861
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         73264 non-null  object 
 1   URL          73264 non-null  object 
 2   Style        73264 non-null  object 
 3   StyleID      73264 non-null  int64  
 4   Size(L)      73264 non-null  float64
 5   OG           73264 non-null  float64
 6   FG           73264 non-null  float64
 7   ABV          73264 non-null  float64
 8   IBU          73264 non-null  float64
 9   Color        73264 non-null  float64
 10  BoilSize     73264 non-null  float64
 11  BoilTime     73264 non-null  int64  
 12  BoilGravity  73264 non-null  float64
 13  Efficiency   73264 non-null  float64
 14  SugarScale   73264 non-null  object 
 15  BrewMethod   73264 non-null  object 
dtypes: float64(9), int64(2), object(5)
memory usage: 9.5+ MB


In [44]:
beers.describe()

,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency
count,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000,73264.000000
mean,59.766420,43.993789,1.406336,1.075667,6.138520,44.381082,13.404036,49.800442,65.096992,1.352552,66.334984
std,56.856388,180.516520,2.198066,0.425391,1.873969,41.606893,11.935438,193.311931,15.011774,1.889688,14.095878
min,1.000000,1.000000,1.000000,-0.003000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,10.000000,18.930000,1.051000,1.011000,5.080000,23.490000,5.170000,20.820000,60.000000,1.040000,65.000000
50%,34.000000,20.820000,1.058000,1.013000,5.790000,35.870000,8.440000,27.440000,60.000000,1.048000,70.000000
75%,109.000000,23.660000,1.069000,1.017000,6.830000,56.490000,16.780000,30.000000,60.000000,1.062000,75.000000
max,176.000000,9200.000000,34.034500,10.341400,54.720000,3409.300000,186.000000,9700.000000,240.000000,52.600000,100.000000
